190715 조진욱 커널연습 - 190807 정리

https://www.youtube.com/watch?v=_iqz7tFhox0&list=PLC_wC_PMBL5MnqmgTLqDgu4tO8mrQakuF&index=1

# 준비

In [0]:
import missingno as msno # 데이터의 null 이 많을 때 찾기 유용하게 해줌
import warnings 
warnings.filterwarnings('ignore') # 특정 필요없은 warning 들 무시

%matplotlib inline 
# 그래프가 그려지면 새로운 창에 생성되는데 이 결과 안에 나올수있도록 하는것. 

In [0]:
plt.style.use('seaborn') # plot 되는 그래프들의 스타일을 seaborn 으로 지정해줌. 여러 스타일 지정가능
sns.set(font_scale=2.5) # 처음 시작할때 font scale 값 자체를 지정해줌

In [0]:
for col in df_train.columns:
    msg = 'column: {:^10} \t Percent of NaN value: {:.2f}%'.format(col, 100 * (df_train[col].isnull().sum() / df_train[col].shape[0]))
    print(msg)
# nan value 비율 보기. 다른 커널에서도 비슷한 식으로 만들었었는데?  
# format string {:^10}중앙 {:>10}오른쪽 {:<10} 왼쪽

3주차 2회 커널 nan 값 보기

total = df_train.isnull().sum().sort_values(ascending = False)
percent = 100* (df_train.isnull().sum() / df_train.isnull().count()).sort_values(ascending=False)
missing_df = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

missing_df.head(20)


-여기랑 거의 비슷한데 여기선 isnull().count() 가지고 해당하는 열의 모든 값 센듯

In [0]:
msno.matrix(df=df_train.iloc[:,:], figsize=(8,8), color=(1, 0.2, 0.2))
# msno.matrix - plotting 까지 진행해줌 


# EDA

In [0]:
f, ax = plt.subplots(1 , 2, figsize=(18, 8))
# figure 안에 plot 을 두개를 만들어줄건데 (1, 2) 즉 1*2 로 만들어주겠음. ax[0], ax[1] 존재. size 는 *** 

df_train['Survived'].value_counts().plot.pie(explode=[0, 0.1], autopct='%1.1f%%', ax=ax[0], shadow=True) 
# .plot.pie 로 보여주기 # explode - 두 wedge 를 얼마나 떨어뜨릴지, 각 개체마다 지정가능해서 리스트로 받음
# autopct 안에있는 값 어떻게 나타낼건지, ax 어떤 파트에 집어넣을지, shadow 그림자 넣을지 말지
# 대부분의 경우 ax= '' 가지고 지정가능. 
sns.countplot

In [0]:
# df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=True).count() # 그룹내 전체 인원
# df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=True).sum() # 생존한 사람
# df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=True).mean() # 생존률(전체 중에 특정 값의 대상)
df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=True).mean().sort_values(by='Survived', ascending=True).plot.bar()
# pclass 를 인덱스로 해둬야 나중에 그림그릴때 편해짐
# dataframe 은 무조건 plot 이 가능


Crosstab
---



In [0]:
# crosstab 으로 target label 에 대한 feature value 들의 분포를 간단하게 볼 수 있음.
pd.crosstab(df_train['Survived'], df_train['Pclass'], margins=True).style.background_gradient(cmap='binary')

In [0]:
# plot bar 과 count plot 비교
df_train['Pclass'].value_counts().plot.bar(color=['darkgreen', 'red', 'blue'], ax=ax[0])

# countplot 의 hue 로 crosstab 같은 기능을 시각화 가능(죽은 사람과 산 사람의 비율같은 것들)
# hue 를 통해 주력인 p class 를 survived 에 맞게 나누어 보여줌
sns.countplot('Pclass', hue = 'Survived', data= df_train, ax = ax[1])

In [0]:
sns.factorplot('Pclass', 'Survived', hue='Sex', data=df_train, aspect=1.5) 
# factor 에 따른 차이를 실선으로 보여줌 aspect 은 선 크기

In [0]:
# 이번엔 Pclass 가지고 플랏 나누기 *이걸 col 대신 hue 쓰면 같은 플랏에 생김
sns.factorplot(x='Sex', y='Survived', col='Pclass', data=df_train, saturation=.5,
              size=3, aspect=1)
# sns.factorplot(x='Sex', y='Survived', hue='Pclass', data=df_train, saturation=.5,
#               size=5, aspect=1) 

In [0]:
print("제일 나이 많은 탑승객 : {:.1f} years".format(df_train['Age'].max())) # 최대 최소 평균값 빠르게 보여주기
print("제일 나이 어린 탑승객 : {:.1f} years".format(df_train['Age'].min()))
print("탑승객 평균 나이: {:.1f} years".format(df_train['Age'].mean()))

In [0]:
fig, ax = plt.subplots(1, 1, figsize = (9, 5)) # kde 플랏은 같은 figure 안에 그리는게 좋음
sns.kdeplot(df_train[df_train['Survived'] == 1]['Age'], ax = ax)
sns.kdeplot(df_train[df_train['Survived'] == 0]['Age'], ax = ax)
plt.legend(['Survived == 1', 'Survived == 0'])

비교 - kdeplot 과 histogram

kde = 커널 밀도 추정
보통 데이터 분포를 이야기 할 때 쓰이는 가우시안 분포를 모델에 맞추거나 데이터의 분포를 그대로 본뜨는(히스토그램)을 이용하는데 히스토그램은 딱딱하니깐 그보다 스무스하게 나오는 밀도 추정을 통해 시각화를 함

[커널밀도추정에 의한 분포근사](https://blog.naver.com/loiu870422/220660847923)


인덱싱 하는 법,슬라이싱//iterrows, cols//특정 조건해당하는 사람 넣기
---




In [0]:
# df_train.iloc[0:3] # 슬라이싱 row col 순으로
# for i in df_train.iterrows(): # row를 자동으로 하나씩 검색. enumerate 와 같음
# df_train[df_train['Survied'] == 1] # survived 가 1 인 대상만 검색

plt 나타내는 3가지 방법

In [0]:
# f = plt.figure(figsize = (10,10)) # axis 없이 도화지를 깔음
# f, ax = plt.subplots(1, 1, figsize = (10, 10)) # axis 를 구분해서 넣을 수 있음(여러 그래프) ax.plot을 마지막에 label 을 ax.set_xlabel
# plt.figure(figsize(10, 10)) # 도화지도 안깔고 바로 플랏. plt.plot 을 마지막에 해줘야함. label 을 plt.xlabel 이용 

In [0]:
df_train['Age'][df_train['Pclass'] == 2].plot(kind='kde') # kde 를 바로 할 수 도 있음. 여기선 age 분포 보고싶어서 

sns.kdeplot(df_train[(df_train['Survived'] == 0) & (df_train['Pclass'] == 1)]['Age'], ax = ax)
# 따로 변수지정안하고 바로 만들기 loc 쓰기가 더 좋긴한데.. 여기서도 age 분포 보고싶었음

어린 친구들이 정말 생존확률이 높은가를 보고 싶음

0-10살, 0-20살 이런식으로 구분해서 

In [0]:
change_age_survival_ratio = []

for i in range(1, 80): # 나이 분포가 1~80 까지 있다고 가정
    change_age_survival_ratio.append(df_train[df_train['Age'] < i]['Survived'].sum() / 
                                     len(df_train[df_train['Age'] < i]['Survived']))
    
plt.figure(figsize = (7, 7))
plt.plot(change_age_survival_ratio)
plt.title("Survival rate chagne depending on range of Age", y = 1.02) 
# y 는 그래프에서 타이틀의 위치를 옮겨줌 1.02 가 적절 

violinplot

In [0]:
sns.violinplot('Pclass', 'Age', hue='Survived', data = df_train, scale='count', split=True, ax=ax[0])
ax[0].set_title('Pclass and Age vs Survived')
ax[0].set_yticks(range(0, 110, 10))
# scale 을 area 로 바꿀시 왼쪽 오른쪽의 면적을 같게 바꿔주므로 둘의 분포 차이를 알기 쉽다
# scale 을 count로 바꿀시 둘의 값을 scaling 해서 보여주므로 실제 값을 알기 좋다. 둘의 전체적인 차이도 알수 있다 
# range 는 min max 그리고 간격으로 나타난다
# ytick 으로 왼쪽 y 축 지표들 바꿀수 있따 


한줄쓰기 끝판왕..
---



In [0]:
f, ax = plt.subplots(figsize = (7, 7))
df_train[['Embarked', 'Survived']].groupby(['Embarked'], as_index=True).mean().sort_values(by='Survived', ascending=False).plot.bar(ax = ax)
# Embarked 와 Survived 열에 대해서  embarked에 따라 group 을 지을 건데 지은 애들은 index로 추가할거야. 그것들은 각 embarked 값에 대한 평균값으로 즉 전체비율로 나타내라
# 그리고 그걸 survived 값에 따라 sort 하는데 ascending이 아니라 descending 이야. 그걸 플롯을 할건데 bar 타입에 할거고 axes 는 ax 에 하렴


hue로 다른 피쳐 섞어주기.  2by2 subplot
---



In [0]:
f, ax = plt.subplots(2, 2, figsize = (12, 10))
sns.countplot('Embarked', data=df_train, ax=ax[0, 0])
ax[0, 0].set_title('(1) No.of Passengers Boarded')

sns.countplot('Embarked', hue='Sex', data=df_train, ax=ax[0, 1])
ax[0, 0].set_title('(2) Male-Female split for embarked')

sns.countplot('Embarked', hue='Survived', data=df_train, ax=ax[1, 0])
ax[1, 0].set_title("(3) Embarked vs Survived")

sns.countplot('Embarked', hue='Pclass', data=df_train, ax=ax[1, 1])
ax[1, 1].set_title("(4) Embarked vs Pclass")

plt.subplots_adjust(wspace=0.5, hspace=0.5)# 좌우 상하 간격 맞춰주는 메서드
plt.show()

In [0]:
df_train[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=True).mean().sort_values(by='Survived', ascending=False).plot.bar(ax=ax[2])
# familysize와 survived 열에 대해 familysize 를 따라 groupby 할건데 index 에 추가할거야. 이걸 familysize 값에 따라 전체 비율을 나타내줘
# 그리고 그걸 survived 값에 따라 sort 하는데 ascending이 아니라 descending 이야. 그걸 플롯을 할건데 bar 타입에 할거고 axes 는 3번째 axes 에 하렴
ax[2].set_title('(3) Survived rate depending on Familysize', y = 1.02)


In [0]:
g = sns.distplot(df_train['Fare'], color='b', label='Skewness: {:.2f}'.format(df_train['Fare'].skew()), ax= ax)
# displot을 fare 값에 대해서 그릴게. 색은 파란색이야. label 로 skewness 어쩌구라고 하면서 뒤에 뭘 달건데 그 값은 df fare 열의 skewness 값을 계산해서 넣어.

왜도와 첨도
왜도나 첨도 가 높으면 머신러닝이 제대로 기능을 못할 수 있음. (이 데이터를 잘 대표할 수 없게 됨)

[관련 내용 블로그](https://blog.naver.com/drwill/221581365327)


여기선 skewness 를 없에는 작업으로서 로그를 취해줌

In [0]:
df_train['Fare']=df_train['Fare'].map(lambda i : np.log(i) if i > 0 else 0)
df_test['Fare']=df_test['Fare'].map(lambda i : np.log(i) if i > 0 else 0)
# Fare 열에 대해서 mapping 을 해서 값을 바꿔줄건데 각 값에 대해서 lambda 함수를 수행해줘
# lambda 함수는 i 입력이 0보다 클시 np.log 를 취해주고 아니면 0을 반환해줘 
# map과 apply 모두 데이터프레임 안의 값을 바꿔주는데 보통 map 은 lambda 이고 apply 는 
# 보통 일반 함수를 사용함.


다시보면 skewness 가 줄어들은 것을 볼 수 있음(feature engineering)

이름을 기준으로 그룹을 나누어 볼거임

In [0]:
df_train['Name'].str.extract # 필요한 스트링 값들을 추출해줌 정규표현식

<bound method StringMethods.extract of <pandas.core.strings.StringMethods object at 0x7ff226e6dc18>>

정규표현식을 넣어서 각 데이터 안에 조건에 해당하는 텍스트를 추출해주는 메서드임

In [0]:
df_train['Initial'] = df_train['Name'].str.extract('([A-Za-z]+)\.')
df_test['Initial'] = df_test['Name'].str.extract('([A-Za-z]+)\.')

In [0]:
temp = df_train['Name'][:10]
temp.str.extract('([A-Za-z]+)\.')

,0
0,Mr
1,Mrs
2,Miss
3,Mrs
4,Mr
5,Mr
6,Mr
7,Master
8,Mrs
9,Mrs


In [0]:
pd.crosstab(df_train['Initial'], df_train['Sex']).T.style.background_gradient(cmap='summer_r')

Initial,Capt,Col,Countess,Don,Dr,Jonkheer,Lady,Major,Master,Miss,Mlle,Mme,Mr,Mrs,Ms,Rev,Sir
Sex,,,,,,,,,,,,,,,,,
female,0,0,1,0,1,0,1,0,0,182,2,1,0,125,1,0,0
male,1,2,0,1,6,1,0,2,40,0,0,0,517,0,0,6,1


별로 없는 애들은 하나로 합치려고함
값을 치환해주는 replace 메서드

In [0]:
df_train['Initial'].replace([a],[b],inplace=True)
# initial 열에서 a를 찾아서 b로 바꿔줘
#inplace 를 빼면 적용이 안되는게 생김


텍스트 replace // 숫자 replace도 가능한가? 
---
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html 

tfidf 바꿀때 이걸 썼으면.. ㅎㅎ


In [0]:
df_test['Initial'].replace(['Mlle', 'Mme', 'Ms', 'Jonkheer', 'Rev', 'Col', 'Lady', 'Countess', 'Dona', 'Capt', 'Don', 'Sir', 'Dr', 'Major'
],['Miss', 'Miss', 'Miss', 'Other', 'Other', 'Other', 'Mrs', 'Mrs', 'Mr', 'Mr', 'Mr', 'Mr', 'Mr', 'Mr'
],inplace=True) #inplace 를 빼면 적용이 안되는게 생김

In [0]:
df_train.groupby('Initial').mean()['Survived']

In [0]:
df_train.loc[(df_train['Age'].isnull()), :] # 'Age'가 비어있는 값을 가진 모든 행을 출력해라

In [0]:
df_train.loc[(df_train['Age'].isnull()) & (df_train['Initial'] == 'Mr'), 'Age'] = 33 # 'age' 가 비어있고 이니셜이 mr 인 데이터의 age 값을 33으로 바꿔라
# train 에 적용하는 모든 feature engineering 은 train과 같은 방식과 같은 파라메터로 테스트 또한 바꿔줘야함

In [0]:
for i in df_train.loc[(df_train['Initial'] == 'Mr'), 'Age'].isnull():
    if i:
        print("Null, NaN data has been found")

print("There are no Null, NaN data!")

# 널데이터 확인하기 
df.isnull().any()


Apply 메서드에 직접만든 함수 적용하기
---



In [0]:
def category_age(x):
    if x < 10:
        return 0
    elif x < 20:
        return 1
    elif x < 30:
        return 2
    elif x < 40:
        return 3
    elif x < 50:
        return 4
    elif x < 60:
        return 5
    elif x < 70:
        return 6
    else:
        return 7


In [0]:
df_train['Age_cat_2'] = df_train['Age'].apply(category_age)

서로 같은 지 확인 . 

In [0]:
(df_train['Age_cat'] == df_train['Age_cat_2']).all() # 모든게 true 이면 true
# .any() 하나라도 true 이면 true

True


스트링 데이터를 숫자로 벡터화하기
---



In [0]:
df_train.Initial.unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Other'], dtype=object)

저번처럼 loc 을 사용해서 일일히  = 어쩌구 해줄 수 도 있지만 그보단 이번엔 매핑을 이용할 것

In [0]:
df_train['Initial'] = df_train['Initial'].map({'Master': 0, 'Miss': 1, 'Mr': 2, 'Mrs': 3, 'Other': 4})
df_test['Initial'] = df_test['Initial'].map({'Master': 0, 'Miss': 1, 'Mr': 2, 'Mrs': 3, 'Other': 4})


Heatmap
---



In [0]:
sns.heatmap(heatmap_data.astype(float).corr(), linewidths=2.0, vmax=1.0,
           square=True, cmap=colormap, linecolor='white', annot=True, annot_kws={'size' : 10}, fmt = '.2f')

# heatmap 쓸건데 데이터는 heatmap_data를 float 으로 데이터타입을 바꾼 것을 corr 한 것을 쓸거야
# line두깨는 저렇게 color map 을 저걸 쓰고 선 색은 저렇게...
# vmax 즉 가장 높은 값의 색이 되 값을 지정 vmin 도 가능
# square ? 
# annotate 는 할건데 kwargs를 따로 지정해줄게 (사이즈 하나), annot 의 포멧은 소숫점 둘째까지

# mask 도 가능하다는거 알아두기 


Categorical data 를 원핫 인코딩으로 만들기
---
카테고리 데이터를 정말 카테고리 데이터 답게 쓸 수 있도록 단순히 0, 1, 2, 3 이 아니라 하나의 벡터 형식으로 바꾸는 것을 말함. 

initial 만 할건데, 현재 클래스가 5개니깐 5개 카테고리 중에 무엇인지? 를 [0 1 0 0] 의 형태로 바꾼다고 생각하면 됨. 

이게 왜 좋냐면, 각각의 정보에 대해서 master 인지 아닌지, miss 인지 아닌지를 정확하게 표현할 수 있음 . 

다른 예로 월드컵 에 참가한 국가들의 순위를 보면 사실 거기서 국가들의 순위 정보를 나타내려면 그것은 서술형 데이터임. 순서를 따지는 게 중요함.

카테고리 데이터(예를들어 월드컵 예선 몇 조)는 순서같은게 아니라 뭐보다 크다, 작다가 큰 의미가 없음 ( 특히 결정 트리를 만들 때는 이런 걸 잘 고려해야함)

12년도 월드컵에 10위 안 이였나?
12년도 월드컵에 A조였나?

둘의 정보량은 분명히 다름

In [0]:
df_train = pd.get_dummies(df_train, columns=['Initial'], prefix='Initial') 
df_test = pd.get_dummies(df_test, columns=['Initial'], prefix='Initial') 
# df_train 안에 initial 컬럼을 지우고 대신 initial_0, 1,.... 로 이어지는 
# 원핫 피쳐컬럼을 붙여줌

In [0]:
df_train = pd.get_dummies(df_train, columns=['Embarked'], prefix='Embarked') 
df_test = pd.get_dummies(df_test, columns=['Embarked'], prefix='Embarked') 

from sklearn.preprocessing import OnehotEncoder 이것도 있음

데이터 프레임 이 아닌 경우 쓰는 방법

원핫은 클래스의 수에 따라 컬럼이 너무 커져서 오히려 문제가 생길 수 있기 때문에 다른 방식으로 인코딩 가능. 


barh 를 통해 bar horizontal 그래프 
---



In [0]:
Series_feat_imp.sort_values(ascending=True).plot.barh()